## Data Collection

> This notebook contains collection of data from cragslist. We collected 923 records. We have used MongoDB for data storage.

### Import libraries

In [60]:
from pymongo import MongoClient
import urllib2
from bs4 import BeautifulSoup

### Connect to Database

In [61]:
client = MongoClient()
db = client['craigslist']
freebies_coll = db['freebies']

### Base urls needs to be Processed

In [56]:
page_url = 'https://chicago.craigslist.org'
list_url = page_url+'/search/zip?s='

### Fetch Items from Cragslist

In [16]:
def get_title(soup_html):
    """Gets Title from html soup object"""
    return soup_html.title.string 

In [29]:
def get_desc(soup_html):
    """Gets Description from soup object"""
    desc = ''
    for s in soup_html.find(id="postingbody").contents:
        desc = desc + ' ' + unicode(s)
    return desc
    #return unicode(soup_html.find(id="postingbody").string)

In [10]:
def get_location(soup_html):
    """Gets Location from soup object"""
    location = {}
    if(soup_html.find(id="map")):
        location['latitude'] = soup_html.find(id="map")['data-latitude']
        location['longitude'] = soup_html.find(id="map")['data-longitude']
        location['accuracy'] = soup_html.find(id="map")['data-accuracy']
    return location

In [11]:
def get_posted_date(soup_html):
    """Gets when the Item was posted"""
    return soup_html.time['datetime']

In [31]:
def get_contents(url):
    """Get Item content like title, description from given url"""
    response = urllib2.urlopen(url)
    html = response.read()
    soup_html = BeautifulSoup(html, 'html.parser')
    content = {}
    content['title'] = get_title(soup_html)
    content['desc'] = get_desc(soup_html)
    if get_location(soup_html):
        content['location'] = get_location(soup_html)
    content['posteddate'] = get_posted_date(soup_html)
    return content

In [57]:
def get_ids(url):
    """Given the list url gets All the Item Ids from it which will be later used to obtain Item Info"""
    ids = []
    response = urllib2.urlopen(url)
    html = response.read()
    soup = BeautifulSoup(html, 'html.parser')
    soup_p = (soup.find_all('p'))
    for p in soup_p:
        ids.append((p.get('data-pid'),p.a['href']))
    return ids

In [69]:
def insert_to_db(key,data):
    freebies_coll.replace_one(key,content,True)

In [70]:
for page in range(0,1100,100):
    for _id,url in get_ids(list_url+str(page)):
        content = get_contents(page_url+str(url))
        #content['_id'] = _id
        insert_to_db({'_id':_id},content)
    print 'cralled %d page'%(page+100)

cralled 100 page
cralled 200 page
cralled 300 page
cralled 400 page
cralled 500 page
cralled 600 page
cralled 700 page
cralled 800 page
cralled 900 page
cralled 1000 page
cralled 1100 page
